In [1]:
import random
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from datasets import load_dataset



d:\HocTap\ChatBot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load a model
from dotenv import load_dotenv
import os
load_dotenv()
hf_token = os.environ["HF_TOKEN"]
model = SentenceTransformer("hiieu/halong_embedding", token=hf_token)

d:\HocTap\ChatBot\myenv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Windows\.cache\huggingface\hub\models--hiieu--halong_embedding. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
# Load the Touche-2020 IR dataset (https://huggingface.co/datasets/BeIR/webis-touche2020, https://huggingface.co/datasets/BeIR/webis-touche2020-qrels)
corpus = load_dataset("GreenNode/zalo-ai-legal-text-retrieval-vn", "corpus", split="corpus", token=hf_token)
queries = load_dataset("GreenNode/zalo-ai-legal-text-retrieval-vn", "queries", split="queries", token=hf_token)
relevant_docs_data = load_dataset("GreenNode/zalo-ai-legal-text-retrieval-vn", "default", split="train", token=hf_token)

In [4]:
# For this dataset, we want to concatenate the title and texts for the corpus
corpus = corpus.map(lambda x: {'text': x['title'] + " " + x['text']}, remove_columns=['title'])

In [5]:
# Shrink the corpus size heavily to only the relevant documents + 5,000 random documents
required_corpus_ids = set(map(str, relevant_docs_data["corpus-id"]))
required_corpus_ids |= set(random.sample(corpus["_id"], k=5_000))
corpus = corpus.filter(lambda x: x["_id"] in required_corpus_ids)

Filter: 100%|██████████| 61425/61425 [00:00<00:00, 122523.29 examples/s]


In [6]:
# Convert the datasets to dictionaries
corpus = dict(zip(corpus["_id"], corpus["text"]))  # Our corpus (cid => document)
queries = dict(zip(queries["_id"], queries["text"]))  # Our queries (qid => question)
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for qid, corpus_ids in zip(relevant_docs_data["query-id"], relevant_docs_data["corpus-id"]):
    qid = str(qid)
    corpus_ids = str(corpus_ids)
    if qid not in relevant_docs:
        relevant_docs[qid] = set()
    relevant_docs[qid].add(corpus_ids)

In [7]:
# Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR metrics.
ir_evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name="hiieu/halong_embedding",
)

In [8]:
corpus_ids = list(corpus.keys())
corpus_texts = list(corpus.values())

# Encode trước
corpus_embeddings = model.encode(
    corpus_texts,
    batch_size=64,  # giảm batch size
    convert_to_tensor=True,
    device="cuda"  # hoặc "cpu"
)

In [9]:
# Đánh giá với embedding đã có
results = ir_evaluator(model, corpus_embeddings=corpus_embeddings)

In [10]:
print(ir_evaluator.primary_metric)
# => "BeIR-touche2020-test_cosine_map@100"
print(results[ir_evaluator.primary_metric])
# => 0.29335196224364596

hiieu/halong_embedding_cosine_ndcg@10
0.8170687010083636


In [11]:
from pprint import pprint
pprint(results)

{'hiieu/halong_embedding_cosine_accuracy@1': 0.7060032894736842,
 'hiieu/halong_embedding_cosine_accuracy@10': 0.9280427631578947,
 'hiieu/halong_embedding_cosine_accuracy@3': 0.8536184210526315,
 'hiieu/halong_embedding_cosine_accuracy@5': 0.8955592105263158,
 'hiieu/halong_embedding_cosine_map@100': 0.7836714683261421,
 'hiieu/halong_embedding_cosine_mrr@10': 0.7860882675438589,
 'hiieu/halong_embedding_cosine_ndcg@10': 0.8170687010083636,
 'hiieu/halong_embedding_cosine_precision@1': 0.7060032894736842,
 'hiieu/halong_embedding_cosine_precision@10': 0.09440789473684211,
 'hiieu/halong_embedding_cosine_precision@3': 0.28824013157894735,
 'hiieu/halong_embedding_cosine_precision@5': 0.1819078947368421,
 'hiieu/halong_embedding_cosine_recall@1': 0.6961348684210527,
 'hiieu/halong_embedding_cosine_recall@10': 0.9225603070175438,
 'hiieu/halong_embedding_cosine_recall@3': 0.8471765350877194,
 'hiieu/halong_embedding_cosine_recall@5': 0.8895970394736842}
